In [ ]:
import numpy as np
from collections import Counter
import mido
from mido import MidiFile, MidiTrack, Message, MetaMessage

In [ ]:
#Dictionaries
note_to_number = {
    "C": 0, "C#": 1, "Db": 1, "D": 2, "D#": 3, "Eb": 3, "E": 4, "F": 5, "F#": 6, "Gb": 6,
    "G": 7, "G#": 8, "Ab": 8, "A": 9, "A#": 10, "Bb": 10, "B": 11
}

number_to_note = {v: k for k, v in note_to_number.items()}

scale_patterns = {
    "chromatic": { #will sound horrible but is the 12-tone basis
        "major": {
            "natural": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 
        },
        "minor": {
            "natural": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
        }
    },
    "heptatonic": {
        "major": {
            "natural": [2, 2, 1, 2, 2, 2, 1], 
            "harmonic": [2, 2, 1, 3, 1, 2, 1],
            "hungarian": [3, 1, 1, 3, 1, 2, 1] #aka double harmonic minor
        },
        "minor": {
            "natural": [2, 1, 2, 2, 1, 2, 2],
            "harmonic": [2, 1, 2, 2, 1, 3, 1],
            "hungarian": [2, 1, 3, 1, 1, 3, 1] #aka double harmonic minor
        }
    },
    "hexatonic": {
        "major": {
            "blues": [2, 1, 1, 3, 2, 3],
            "whole_tone": [2, 2, 2, 2, 2, 2] #will sound horrible (same as minor whole_tone)
        },
        "minor": {
            "blues": [3, 2, 1, 1, 3, 2],
            "whole_tone": [2, 2, 2, 2, 2, 2] #will sound horrible (same as major whole_tone)
        }
    },
    "pentatonic": {
        "major": {
            "natural": [2, 2, 3, 2, 3]
        },
        "minor": {
            "natural": [3, 2, 2, 3, 2]
        }
    }
}

In [ ]:
def generate_scale(root_note, scale_category, scale_type, scale_structure):
    # Navigate through the hierarchical structure
    if scale_category not in scale_patterns:
        raise ValueError(f"Scale category '{scale_category}' is not supported.")
    
    if scale_type not in scale_patterns[scale_category]:
        raise ValueError(f"Scale type '{scale_type}' is not supported within '{scale_category}'.")
    
    selected_scale = scale_patterns[scale_category][scale_type]
    
    # If the scale type contains modes (e.g., natural, harmonic, etc.), check for the mode
    if isinstance(selected_scale, dict):
        if scale_structure not in selected_scale:
            raise ValueError(f"Mode '{scale_structure}' is not supported within '{scale_type}'.")
        scale_pattern = selected_scale[scale_structure]
    else:
        scale_pattern = selected_scale  # Handle scales without modes
    
    # Proceed with generating the scale based on root note and intervals
    root_number = note_to_number[root_note]
    scale_numbers = [root_number]
    current_number = root_number
    for step in scale_pattern:
        current_number = (current_number + step) % 12
        scale_numbers.append(current_number)
    
    return scale_numbers

In [ ]:
def find_duplicate(arr):
    counts = Counter(arr)
    return [item for item, count in counts.items() if count > 1]

In [ ]:
#Function to generate all modes for a given scale
def generate_modes(scale_numbers):
    modes = []
    num_notes = len(scale_numbers)-1
    for i in range(num_notes):
        #Create a mode by rotating the scale numbers
        if i==0:
            mode = scale_numbers[i:] + scale_numbers[:i]
            num_to_replace = find_duplicate(mode)[0]
            modes.append(mode)
        else:
            mode = scale_numbers[i:] + scale_numbers[:i]
            #Deal with duplicate scale ending in 1st mode
            value_first = mode[0]
            for note in range(len(mode)):
                if mode[note] == num_to_replace:
                    break_1 = mode[:note+1]
                    break_2 = mode[note+1:]
                    cycled_break = break_2[1:] + break_2[:1]
                    first_value=break_1[0]
                    cycled_break[-1]=first_value
                    break
            mode=break_1+cycled_break
            modes.append(mode)
            #print(mode)
    return modes

In [ ]:
def print_modes(root_note, scale_category, scale_type, scale_structure):
    # Generate the scale using the updated generate_scale function
    scale_numbers = generate_scale(root_note, scale_category, scale_type, scale_structure)
    scale_notes = [number_to_note[num] for num in scale_numbers]
    print(f"\nOriginal {root_note} {scale_structure.capitalize()} {scale_type.capitalize()} Scale: {scale_notes} -> {scale_numbers}\n")
    
    # Generate all modes
    modes = generate_modes(scale_numbers)
    
    # Print each mode
    for i, mode_numbers in enumerate(modes):
        mode_notes = [number_to_note[num] for num in mode_numbers]
        print(f"Mode {i + 1}: {mode_notes} -> {mode_numbers}")
    
    return modes

In [ ]:
# Function to convert a number to Roman numeral
#def number_to_roman(number):
#    roman_numerals = {
#        1: "I", 2: "II", 3: "III", 4: "IV", 5: "V", 6: "VI", 7: "VII"
#    }
#    return roman_numerals.get(number, str(number))

In [ ]:
# Simplified function to generate chords from a mode without Roman numerals or chord quality determination
def generate_chords_from_mode(mode_notes, mode_numbers):
    chords = []
    
    for i in range(len(mode_numbers) - 1):  # Exclude the octave note
        root = mode_numbers[i]
        third = mode_numbers[(i + 2) % (len(mode_numbers) - 1)]
        fifth = mode_numbers[(i + 4) % (len(mode_numbers) - 1)]

        chord_notes = [mode_notes[i], number_to_note[third], number_to_note[fifth]]
        chord_numbers = [root, third, fifth]
        
        chords.append((chord_notes, chord_numbers))
    
    return chords

In [ ]:
# Function to print the chords in the desired format
def print_chords(roman_numerals, chords):
    print("\nChords for the selected mode:")
    print("-".join(roman_numerals))  # Print the Roman numerals

    for chord_notes, chord_numbers in chords:
        print(f"{chord_notes} -> {chord_numbers}")

In [ ]:
def add_key_signature(track, key, scale_type="major"):
    if scale_type == "minor":
        key += 'm'  # Append 'm' for minor keys

    # Add the key signature to the track
    track.append(MetaMessage('key_signature', key=key))

In [ ]:
# Funcxtion to transpose notes up by a given offset and ensure they stay within the valid range (0-127)
#def transpose_up(note, offset):
#    transposed_note = (note + offset) % 128  # Ensure MIDI note values stay within valid range
#    # If the note is above the allowed range, reduce by octaves until it is valid
#    while transposed_note > 127:
#        transposed_note -= 12
#    # If the note is below the allowed range, increase by octaves until it is valid
#    while transposed_note < 0:
#        transposed_note += 12
#    return transposed_note

In [ ]:
# Function to transpose notes up by a given offset and ensure they stay within the valid range (0-127)
def transpose_up(note, offset):
    transposed_note = note + offset  # Simply add the offset
    
    # If the note exceeds 127, bring it down by octaves
    while transposed_note > 127:
        transposed_note -= 12
    # If the note is below 0, bring it up by octaves
    while transposed_note < 0:
        transposed_note += 12
    
    return transposed_note

In [ ]:
def adjust_note(prev_note, next_note, threshold=9):
    difference = abs(next_note - prev_note)
    octave_shift = 12
    
    if difference >= threshold:
        # If the next note is more than 10 higher, reduce its value
        if next_note > prev_note:
            next_note -= octave_shift
        # If the next note is more than 10 lower, increase its value
        else:
            next_note += octave_shift
    
    return next_note

In [ ]:
def ensure_root_is_lowest(root, third, fifth):
    # Ensure the third and fifth are higher than the root
    while third <= root:
        third += 12  # Shift the third up by an octave if it's not higher than the root
    while fifth <= third:
        fifth += 12  # Shift the fifth up by an octave if it's not higher than the third
    
    return [root, third, fifth]

In [ ]:
# Function to adjust the root, 3rd, and 5th if the difference with the previous root exceeds the threshold
def adjust_chord(prev_root, root, third, fifth, threshold=9):
    difference = abs(root - prev_root)
    octave_shift = 12
    
    # If the root difference exceeds the threshold, adjust the root
    if difference >= threshold:
        # If the next root is higher than the previous root, shift it down an octave
        if root > prev_root:
            root -= octave_shift
            third -= octave_shift
            fifth -= octave_shift
        # If the next root is lower than the previous root, shift it up an octave
        else:
            root += octave_shift
            third += octave_shift
            fifth += octave_shift
    
    # Ensure the root is the lowest note after adjustments
    return ensure_root_is_lowest(root, third, fifth)

In [ ]:
# Prompt user for scale category
print("Scale Category Selection:")
for category in scale_patterns:
    print(f"    {category}")
scale_category = input('Enter a scale category from "Scale Category Selection": ').strip().lower()

if scale_category not in scale_patterns:
    print(f"\nScale category '{scale_category}' is not supported.")
else:
    # Prompt user for specific scale within the chosen category
    print(f"\nScale Type Selection within '{scale_category}':")
    
    # Check if the scale category contains a dictionary or directly a list
    if isinstance(scale_patterns[scale_category], dict):
        # Iterate over the scale types (keys) in the selected category
        for scale_type in scale_patterns[scale_category].keys():
            print(f"    {scale_type}")
        scale_type = input('Enter a scale type from "Scale Type Selection": ').strip().lower()
        
        if scale_type not in scale_patterns[scale_category]:
            print(f"\nScale type '{scale_type}' is not supported.")
        else:
            # Handle the inner dictionary or directly access the scale pattern
            selected_scale = scale_patterns[scale_category][scale_type]
            
            # If the selected scale is a dictionary (e.g., "natural", "harmonic" in major), handle accordingly
            if isinstance(selected_scale, dict):
                print(f"\nScale structure Selection within '{scale_type}':")
                for scale_structure in selected_scale.keys():
                    print(f"    {scale_structure}")
                scale_structure = input('Enter a scale structure from "Structure Selection": ').strip().lower()
                
                if scale_structure not in selected_scale:
                    print(f"\nMode '{scale_structure}' is not supported.")
                else:
                    # Proceed with the selected scale and mode
                    scale_intervals = selected_scale[scale_structure]
                    print(f"\nSelected scale intervals: {scale_intervals}")

                    # Prompt user to select a key after selecting the mode
                    print("\nKey Selection:")
                    for key in note_to_number:
                        print(f"    {key}")
                    key = input('Enter a key from "Key Selection": ').strip()

                    if key not in note_to_number:
                        print(f"\nKey '{key}' is not supported.")
                    else:
                        # Print all modes for the user-specified scale type and key
                        modes = print_modes(key, scale_category, scale_type, scale_structure)
                        
                        # Prompt user to select a mode number
                        mode_number = input(f'\nEnter the mode number (1 to {len(modes)}) to generate chords: ').strip()

                        try:
                            mode_number = int(mode_number)
                            if 1 <= mode_number <= len(modes):
                                selected_mode_notes = [number_to_note[num] for num in modes[mode_number - 1]]
                                selected_mode_numbers = modes[mode_number - 1]

                                # Generate and print the chords for the selected mode
                                chords = generate_chords_from_mode(selected_mode_notes, selected_mode_numbers)
                                
                                print("\nChords for the selected mode:")
                                for chord_notes, chord_numbers in chords:
                                    print(f"{chord_notes} -> {chord_numbers}")
                            else:
                                print(f"\nInvalid mode number. Please enter a number between 1 and {len(modes)}.")
                        except ValueError:
                            print("\nInvalid input. Please enter a numerical value.")
            else:
                # If it's directly a scale, use it
                scale_intervals = selected_scale
                print(f"\nSelected scale intervals: {scale_intervals}")
    else:
        # If the category contains a list directly
        print(f"\nSelected scale intervals: {scale_patterns[scale_category]}")

        # Key selection for directly accessed scale
        print("\nKey Selection:")
        for key in note_to_number:
            print(f"    {key}")
        key = input('Enter a key from "Key Selection": ').strip()

        if key not in note_to_number:
            print(f"\nKey '{key}' is not supported.")
        else:
            # Print all modes for the user-specified scale type and key
            modes = print_modes(key, scale_category, scale_type, scale_structure)
            
            # Prompt user to select a mode number
            mode_number = input(f'\nEnter the mode number (1 to {len(modes)}) to generate chords: ').strip() - 1

            try:
                mode_number = int(mode_number)
                if 1 <= mode_number <= len(modes):
                    selected_mode_notes = [number_to_note[num] for num in modes[mode_number - 1]]
                    selected_mode_numbers = modes[mode_number - 1]

                    # Generate and print the chords for the selected mode
                    chords = generate_chords_from_mode(selected_mode_notes, selected_mode_numbers)
                    
                    print("\nChords for the selected mode:")
                    for chord_notes, chord_numbers in chords:
                        print(f"{chord_notes} -> {chord_numbers}")
                else:
                    print(f"\nInvalid mode number. Please enter a number between 1 and {len(modes)}.")
            except ValueError:
                print("\nInvalid input. Please enter a numerical value.")


In [ ]:
# Define separate octave offsets for scale, chords, and drone
SCALE_OCTAVE_OFFSET = 12 * 5  # Scale transposed up by three octaves
CHORD_OCTAVE_OFFSET = 12 * 4  # Chords transposed up by two octaves
DRONE_OCTAVE_OFFSET = 12 * 4  # Drone transposed up by one octave

# Define programs and channels
GRAND_PIANO_PROGRAM = 0  # Grand Piano
NYLON_GUITAR_PROGRAM = 25 #Acoustic Guitar (nylon)
STRING_ENSEMBLE_PROGRAM = 48 #String Ensemble 1
CELLO_PROGRAM = 42  # Cello
BRASS_SECTION_PROGRAM = 61 #Brass Section
PAD_4_PROGRAM = 91 #Pad 4 (choir)
FX_5_PROGRAM = 100 #FX 5 (brightness)
SCALE_CHANNEL = 0
CHORD_CHANNEL = 1 
DRONE_CHANNEL = 2

# Create a new MIDI file and add tracks for scale, chords, and drone
midi_file = MidiFile()
scale_track = MidiTrack(name="Scale")
chord_track = MidiTrack(name="Chords")
drone_track = MidiTrack(name="Drone")
midi_file.tracks.append(scale_track)
midi_file.tracks.append(chord_track)
midi_file.tracks.append(drone_track)

add_key_signature(scale_track, key, scale_type)

# Time variables
quarter_note_duration = 480  # Duration of a quarter note in ticks
half_note_duration = quarter_note_duration * 2
whole_note_duration = quarter_note_duration * 4
scale_duration = quarter_note_duration
chord_duration = quarter_note_duration
drone_duration = quarter_note_duration

In [ ]:
# Define scale and chords
scale_notes = selected_mode_notes + selected_mode_notes[::-1][1:]  # Ascend then descend except first repeat

# Chords for the scale
ascending_chords = chords
octave_chord = [chords[0]]
descending_chords = chords[::-1]
chord_progression = ascending_chords + octave_chord + descending_chords

print(scale_notes)

In [ ]:
# The Scale: Play each note with a consistent octave shift
scale_track.append(Message('program_change', program=NYLON_GUITAR_PROGRAM, channel=SCALE_CHANNEL))

prev_midi_note = None  # Initialize the previous note as None

for note in scale_notes:
    midi_note = transpose_up(note_to_number[note], SCALE_OCTAVE_OFFSET)
    
    if prev_midi_note is not None:
        # Adjust the current note based on the previous note to prevent large jumps
        midi_note = adjust_note(prev_midi_note, midi_note)
    
    scale_track.append(Message('note_on', note=midi_note, velocity=42, time=0, channel=SCALE_CHANNEL))
    scale_track.append(Message('note_off', note=midi_note, velocity=42, time=scale_duration, channel=SCALE_CHANNEL))
    
    prev_midi_note = midi_note  # Update the previous note
    
#print(scale_track)

In [ ]:
#for note in scale_notes:
#    midi_note = transpose_up(note_to_number[note], SCALE_OCTAVE_OFFSET)
#    print(f"Note: {note}, MIDI Note: {midi_note}")  # Debugging print

In [ ]:
# The Chords: Transpose the chord notes using the CHORD_OCTAVE_OFFSET and adjust them accordingly
chord_track.append(Message('program_change', program=GRAND_PIANO_PROGRAM, channel=CHORD_CHANNEL))

prev_root = None  # Initialize previous root note

for i, (chord_notes, _) in enumerate(chord_progression):
    # Use the same note from the scale for the root, transposed down two octaves
    scale_note = scale_notes[i]  # Get the corresponding note from the scale
    root = transpose_up(note_to_number[scale_note], CHORD_OCTAVE_OFFSET)  # Use scale root, two octaves down

    # Transpose the third and fifth using the CHORD_OCTAVE_OFFSET
    third = transpose_up(note_to_number[chord_notes[1]], CHORD_OCTAVE_OFFSET)
    fifth = transpose_up(note_to_number[chord_notes[2]], CHORD_OCTAVE_OFFSET)
    
    if prev_root is not None:
        # Adjust the chord root, third, and fifth based on the previous root and threshold
        adjusted_chord = adjust_chord(prev_root, root, third, fifth)
    else:
        # For the first chord, ensure the root is the lowest note
        adjusted_chord = ensure_root_is_lowest(root, third, fifth)

    # Turn on all chord notes at the same time (time=0 for all)
    for chord_note in adjusted_chord:
        chord_track.append(Message('note_on', note=chord_note, velocity=64, time=0, channel=CHORD_CHANNEL))
    
    # Turn off all chord notes at the same time (first `note_off` gets `chord_duration`, the rest get `time=0`)
    chord_track.append(Message('note_off', note=adjusted_chord[0], velocity=64, time=chord_duration, channel=CHORD_CHANNEL))
    chord_track.append(Message('note_off', note=adjusted_chord[1], velocity=64, time=0, channel=CHORD_CHANNEL))
    chord_track.append(Message('note_off', note=adjusted_chord[2], velocity=64, time=0, channel=CHORD_CHANNEL))
    
    # Set the current root as the previous root for the next iteration
    prev_root = adjusted_chord[0]
    
#print(chord_track)

In [ ]:
drone_track.append(Message('program_change', program=CELLO_PROGRAM, channel=DRONE_CHANNEL))

drone_note = transpose_up(note_to_number[selected_mode_notes[0]], DRONE_OCTAVE_OFFSET)  # First note of the mode
drone_track.append(Message('note_on', note=drone_note, velocity=64, time=0, channel=DRONE_CHANNEL))
drone_duration = quarter_note_duration * len(scale_notes)  # Drone lasts for the entire scale + chords duration
drone_track.append(Message('note_off', note=drone_note, velocity=64, time=drone_duration, channel=DRONE_CHANNEL))

#print(drone_track)

In [ ]:
# Save the midi file
output_file = f'{key}_{scale_category}_{scale_structure}_{scale_type}_mode_{mode_number}.mid'
midi_file.save(output_file)

print(f"MIDI file '{output_file}' has been generated.")